# ** selbstfahrendes Autospiel mit ROS2 und SuperTuxKart **

Willkommen in unserem coolen Projekt!Wir werden in einem Spiel namens SuperTuxKart mit einer Robotersoftware namens ROS2 ein Autofahren für sich selbst machen.

## ** Was werden wir tun **
1. Lerne ein wenig über ROS2
2. Richten Sie unseren Computer ein
3. Starten Sie SuperTuxKart mit ROS2
4. Lassen Sie das Auto die Straße sehen
5. Machen Sie das Auto selbst fahren
6. Beobachten Sie das Auto -Rennen von selbst!

## ** 1.Über Ros2 ** lernen

ROS2 ist eine spezielle Software, die Robotern hilft.Wissenschaftler und Ingenieure verwenden es, damit Roboter die Dinge wie sie sehen, hören, bewegen und die Welt verstehen.

## ** 2.Einrichten unseres Computers **

Wir müssen zuerst eine Software installieren:

1. ** ROS2 **: Befolgen Sie die Anweisungen [here](https://docs.ros.org/en/humble/Installation.html).
2. ** SuperTuxKart installieren **: Laden Sie es aus [here](https://sourceforge.net/projects/maskor/files/stk/ros-humble-supertuxkart.deb) herunter und installieren Sie es.

Öffnen Sie nach der Installation ein Terminal und geben Sie dies ein, um das Spiel mit ROS2 zu starten:

```bash
ros2 run supertuxkart supertuxkart
```

## **3.SuperTuxKart mit ROS2 ** beginnen

Überprüfen Sie, ob alles funktioniert.Öffnen Sie ein Terminal und Typ:

```bash
ros2 node list
```

Sie sollten so etwas wie `/stk_node` sehen.Um mehr darüber zu erfahren, geben Sie: Typ:

```bash
ros2 node info /stk_node
```

Dies zeigt Ihnen, was das Spiel mit ROS2 teilt.

## ** 4.Das Auto die Straße sehen **

Wir werden etwas namens HSV -Filter verwenden, um dem Auto zu helfen, die Straße im Spiel zu sehen.Mal sehen, wie es funktioniert.

### ** Code Erläuterung **

Dieser Code erhält das Spielbild und hilft dem Auto, die Straße zu sehen.

In [ ]:
import rclpy
from sensor_msgs.msg import Image
import cv2
from cv_bridge import CvBridge
import numpy as np

def nothing(x):
    pass

rclpy.init()
node = rclpy.create_node("get_box_color")
initial_flag = True

def hsv_calc(msg):
    global initial_flag
    if initial_flag:
        initial_flag = False
        cv2.namedWindow("Trackbars")
        cv2.createTrackbar("lh", "Trackbars", 0, 179, nothing)
        cv2.createTrackbar("ls", "Trackbars", 0, 255, nothing)
        cv2.createTrackbar("lv", "Trackbars", 0, 255, nothing)
        cv2.createTrackbar("uh", "Trackbars", 179, 179, nothing)
        cv2.createTrackbar("us", "Trackbars", 255, 255, nothing)
        cv2.createTrackbar("uv", "Trackbars", 255, 255, nothing)

    frame = CvBridge().imgmsg_to_cv2(msg, "bgr8")
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    lh = cv2.getTrackbarPos("lh", "Trackbars")
    ls = cv2.getTrackbarPos("ls", "Trackbars")
    lv = cv2.getTrackbarPos("lv", "Trackbars")
    uh = cv2.getTrackbarPos("uh", "Trackbars")
    us = cv2.getTrackbarPos("us", "Trackbars")
    uv = cv2.getTrackbarPos("uv", "Trackbars")

    l_blue = np.array([lh, ls, lv])
    u_blue = np.array([uh, us, uv])
    mask = cv2.inRange(hsv, l_blue, u_blue)
    result = cv2.bitwise_or(frame, frame, mask=mask)

    cv2.imshow("mask", mask)
    cv2.imshow("result", result)
    key = cv2.waitKey(1)
    if key == 27:
        cv2.destroyAllWindows()
        rclpy.shutdown()

node.create_subscription(Image, "/stk_image", hsv_calc, 1)
rclpy.spin(node)
rclpy.shutdown()

Der Code macht dies:

1. Lädt wichtige Bibliotheken für das Projekt.
2. Erstellt eine Funktion "Nichts", um Schieberegler zu machen.
3. Startet ROS2 und erstellt einen Knoten namens `get_box_color`.
4. Richtet Schieberegler so ein, dass die Farben beim ersten Mal ausgeführt werden.
5. Holt das Spielbild und ändert es in ein Farbformat namens HSV.
6. Verwendet die Slider -Werte, um eine Maske zu erstellen, die die Straßenfarbe findet.
7. zeigt die Maske und die Straße, die von der Maske gefunden wurde.
8. Hört die Spielbilder zu und führt die Funktion für jedes Bild aus.

## ** 5.Das Auto selbst fahren **

Jetzt werden wir den HSV -Filter verwenden, um das Auto selbst der Straße zu folgen.

### ** Code Erläuterung **

Dieser Code hilft dem Auto, die Straße zu sehen und dann das Auto zu kontrollieren, um auf der Straße zu bleiben.

In [ ]:
import rclpy
from rclpy.node import Node

from sensor_msgs.msg import Image
import cv_bridge
import cv2
from geometry_msgs.msg import Twist
import numpy as np

CONTROL_COEFFICIENT = 0.02


class LaneFollower(Node):
    def __init__(self):
        super().__init__("my_running_node")
        self.create_subscription(
            Image, "/stk_image", self.image_cb, 1
        )
        self.cmd_pub = self.create_publisher(Twist, "/cmd_vel", 1)

    def image_cb(self, msg):
        img = cv_bridge.CvBridge().imgmsg_to_cv2(msg, desired_encoding="bgr8")

        twist_msg = Twist()
        twist_msg.linear.x = 0.3

        img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        img = cv2.GaussianBlur(img, (5, 5), 0)
        mask = cv2.inRange(img, np.array([92, 0, 90]), np.array([119, 42, 154]))

        # Find the largest segmented contour
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

        if contours:
            largest_contour = max(contours, key=cv2.contourArea)
            largest_contour_center = cv2.moments(largest_contour)

            if largest_contour_center["m00"] != 0:
                center_x = int(
                    largest_contour_center["m10"] / largest_contour_center["m00"]
                )
                # Find error (the lane distance from the target distance)
                error = center_x - img.shape[1] / 2
                twist_msg.angular.z = -error * CONTROL_COEFFICIENT
        else:
            twist_msg.linear.x = 0.0
            twist_msg.angular.z = 0.0

        self.cmd_pub.publish(twist_msg)


def main():
    rclpy.init()
    rclpy.spin(LaneFollower())
    rclpy.shutdown()


if __name__ == "__main__":
    main()

Der Code macht dies:

1. Lädt wichtige Bibliotheken für das Projekt.
2. Erstellt eine Funktion "Nichts", um Schieberegler zu machen.
3. Startet ROS2 und erstellt einen Knoten namens "autonomous_drive".
4. Erstellt eine Möglichkeit, Antriebsbefehle an das Auto zu senden.
5. Richten Sie Schieberegler so ein, dass die Farben beim ersten Mal die Funktion anpassen.
6. Holt das Spielbild und ändert es in ein Farbformat namens HSV.
7. Verwendet die Slider -Werte, um eine Maske zu erstellen, die die Straßenfarbe findet.
8. zeigt die Maske und die Straße, die von der Maske gefunden wurde.
9. Findet das Straßenzentrum.
10. Berechnet berechnet, inwieweit die Straßenmitte von der Mitte des Bildes entfernt ist.
11. macht einen Befehl, das Auto in Richtung der Straße zu steuern.
12. Sendet den Befehl an das Auto.
13. Hört die Spielbilder zu und führt die Funktion für jedes Bild aus.

## ** 6.Das letzte autonome Rennen laufen **

Jetzt, da alles eingerichtet ist, lass uns das letzte autonome Rennen durchführen!Starten Sie SuperTuxKart und beobachten Sie, wie Ihr Auto selbst um die Strecke fährt.